# ニューラルネットワーク学習処理

## 復習｜勾配計算

まずは、シンプルなニューラルネットワークを構築し、正しく勾配計算されるか確認する

In [1]:
"""
@test: 参考書との比較（正しく勾配計算されるか確認）
"""
# ニューラルネットワーク実装読み込み
include("./lib/Neuron.jl")
include("./lib/Functions.jl")

# 数値微分による勾配計算
## 関数(Array{Float64,2})::Float64, 入力値::Array{Float64,2} -> 勾配::Array{Float64,2}
numeric_gradient(f, x::Array{Float64,2})::Array{Float64,2} = begin
    h = 1e-4 # 10^(-4)
    grad = Array{Float64, 2}(undef, size(x, 1), size(x, 2)) # xと同じ次元の行列を生成
    # 各変数ごとの数値微分を行列にまとめる
    for row in 1:size(x, 1), col in 1:size(x, 2)
        # 指定indexの変数に対する中心差分を求める
        org = x[row, col]
        x[row, col] = org + h
        f1 = f(x) # f([..., x[row, col] + h, ...]) -> Float64
        x[row, col] = org - h
        f2 = f(x) # f([..., x[row, col] - h, ...]) -> Float64
        grad[row, col] = (f1 - f2) / 2h # (row, col)番目の変数に対する数値微分
        x[row, col] = org # x[i]の値をもとに戻す
    end
    return grad
end
    
# シンプルなニューラルネットワーク
SimpleNet() = Network(1,
    [
        zeros(1, 3) # 1 x 3 Array{Float64,2} bias_1 [0 0 0]
    ],
    [
        zeros(2, 3) # 2 x 3 Array{Float64,2} weight_1 [0 0 0; 0 0 0]
    ]
)

# 活性関数を使わず、ソフトマックス関数で出力するだけの推論処理
predict(net::Network, x::Array{Float64,2})::Array{Float64,2} = softmax(x * net.w[1] + net.b[1])

# 損失関数: 交差エントロピー誤差
loss(net::Network, x::Array{Float64,2}, t::Array{Float64,2})::Float64 = cross_entropy_error(predict(net, x), t)

x = [0.6 0.9]
t = [0.0 0.0 1.0]

net = SimpleNet()

# 勾配計算
## 2 x 3 Array{Float64,2} [0.2 0.2 -0.4; 0.3 0.3 -0.6] になればOK
dW = numeric_gradient(w->loss(net, x, t), net.w[1])

2×3 Array{Float64,2}:
 0.2  0.2  -0.4
 0.3  0.3  -0.6

## 学習アルゴリズムの実装

これまでに実装した「損失関数」「ミニバッチ」「勾配」「勾配降下法」をまとめることで、ニューラルネットワークの学習アルゴリズムを実装することができる

### 確率的勾配降下法
ニューラルネットワークの学習手順は以下のようなものが基本となる

1. ミニバッチ
    - 訓練データからランダムに一部のデータを選び出す（ミニバッチ）
    - 一回の学習においては、このミニバッチの損失関数の値を減少させることを目的とする
2. 勾配
    - ミニバッチの損失関数を減らすために、各重みパラメータの勾配を算出する
    - 勾配は、損失関数の値を最も減らす方向を示す
3. パラメータの更新
    - 重みパラメータを勾配方向に微小量だけ更新する
4. 1.に戻って同様の手順を繰り返す

ここで、使用する訓練データをミニバッチとして無作為に選び出していることから、このような学習方法を**確率的勾配降下法**（Stochastic Gradient Descent）と呼ぶ

ディープラーニングの多くのフレームワークでは、確率的勾配降下法の頭文字をとって**SGD**という名前の関数で実装されているのが一般的である

### 2層ニューラルネットワークの実装
今回は、手書き数字の学習を行うためのニューラルネットワークとして、2層ニューラルネットワーク（隠れ層1つのニューラルネットワーク）を実装することにする

ネットワーク設計は以下の通りとする

- 入力層:
    - 手書き数字の画像データ（サイズ: 28x28）
    - ニューロン数: 28 * 28 = 784
    - 各ニューロンの入力値は 0.0〜1.0 の実数型である必要がある
- 中間層（隠れ層）1:
    - ニューロン数: 100
    - 活性化関数: シグモイド関数
- 出力層:
    - ニューロン数: 10（0〜9の数字クラスに分類するため）
    - 活性化関数: ソフトマックス関数
    - 損失関数: 交差エントロピー関数

In [2]:
"""
2層ニューラルネットワークによる手書き数字の学習
"""
# Network構造体を継承して2層ニューラルネットワーク実装
TwoLayerNetwork(weight_init_std::Float64=0.01) = Network(2,
    [
        zeros(Float64, 1, 100), # 1x100-Array{Float64,2} バイアス_1: 0行列
        zeros(Float64, 1, 10),  # 1x10-Array{Float64,2} バイアス_2: 0行列
    ],
    [
        rand(UInt8, 784, 100) * weight_init_std, # 784x100-Array{Float64,2} 重み_1: 任意整数 * weight_init_std の乱数行列
        rand(UInt8, 100, 10) * weight_init_std, # 100x10-Array{Float64,2} 重み_2: 任意整数 * weight_init_std の乱数行列
    ]
)

# 推論処理
## Network構造体, 入力信号 -> 出力信号 y
predict(network::Network, x::Array{Float64,2})::Array{Float64,2} = predict(network, sigmoid, softmax, x)

# 推論処理＋損失関数
## Network構造体, 入力信号, 教師データ -> 交差エントロピー誤差
loss(network::Network, x::Array{Float64,2}, t::Array{Float64,2})::Float64 = cross_entropy_error(predict(network, x), t)

# 各パラメータの勾配計算
## Network構造体, 入力信号, 教師データ -> 各パラメータの勾配行列をまとめた辞書
numeric_gradient(network::Network, x::Array{Float64,2}, t::Array{Float64,2})::Dict{AbstractString, Array{Float64,2}} = begin
    loss_func = w -> loss(network, x, t)
    Dict(
        "B1" => numeric_gradient(loss_func, network.b[1]),
        "B2" => numeric_gradient(loss_func, network.b[2]),
        "W1" => numeric_gradient(loss_func, network.w[1]),
        "W2" => numeric_gradient(loss_func, network.w[2]),
    )
end

"""
@test: 2層ニューラルネットワークの推論処理
"""
# ダミー入力データ: 0.0〜1.0 の784サイズデータ 100枚分
x = rand(Float64, 100, 784)

# 推論実行
net = TwoLayerNetwork()
y = predict(net, x)

# 各行ごとに列の合計値が1になっているか確認（softmax関数の特性の確認）
[sum(y[row, :]) for row in 1:size(y, 1)]

100-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [3]:
"""
@test: 2層ニューラルネットワークの勾配計算
"""
# ダミー教師データ: 0.0〜1.0 の10サイズデータ 100枚分
y = rand(Float64, 100, 10)

# 勾配計算
## @timeマクロで時間計測してみると分かるが、ニューロンの数だけ勾配計算する今の方法では非常に多くの時間がかかる
## => Intel(C) Core i7-7700 3.6 GHz で 60〜80秒程度かかる
## => この部分の高速化（誤差逆伝搬法）については後述する
grad = @time numeric_gradient(net, x, y)

 75.332291 seconds (148.44 M allocations: 107.002 GiB, 8.60% gc time)


Dict{AbstractString,Array{Float64,2}} with 4 entries:
  "W2" => [-0.150111 0.516357 … -4.31865e-6 -0.0843425; -0.150111 0.516357 … -4…
  "B2" => [-0.150111 0.516357 … -4.31865e-6 -0.0843425]
  "B1" => [0.0 0.0 … 0.0 0.0]
  "W1" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0…

#### ミニバッチ学習の実装
上記で実装したTwoLayerNetworkに対して、MNISTデータセットを使ってミニバッチ学習を実装する

今回は、ミニバッチサイズを100として、毎回60,000個の訓練データからランダムに100個のデータを抜き出して学習することにする

この100個のミニバッチを対象に勾配を求め、確率的勾配降下法（SGD）によりパラメータを更新する

さらにパラメータ更新を10,000回繰り返し、損失関数の値の推移をグラフで表す

In [4]:
# MLDatasetsパッケージのMNISTSデータセットを使う
using MLDatasets

# 訓練用画像データと教師データをロード
## train_x: 特徴量＝<画像データ｜28x28 グレースケール画像 60,000枚>{28x28x60000 Array{UInt8, 3}}
## train_y: 目的変数＝<数値クラス｜[0..9]の数値 60,000個>{60000 Array{Int, 1}}
train_x, train_y = MNIST.traindata()

# 学習データをニューラルネットワーク用に前処理
train_x = Array{Float64,3}(reshape(train_x, 1, 28*28, :)) # 1 x 784 x 60000 Array{Float64,3}
train_x = permutedims(train_x, [3, 2, 1]) # 60000 x 784 x 1 Array{Float64,3}
train_x = Array{Float64,2}(reshape(train_x, :, 784)) # 60000 x 784 Array{Float64,2}

# 教師データをone-hot-vector形式に変換
train_y = hcat([[i-1 == y ? 1.0 : 0.0 for i in 1:10] for y in train_y]...)' # 60000 x 10 Array{Float64,2}

# 損失関数の履歴
train_loss_list = []

# ハイパーパラメータ
iters_num = 1 # パラメータ更新回数
train_size = size(train_x, 1) # 学習データ枚数: 60,000
batch_size = 100 # ミニバッチサイズ
learning_rate = 0.1 # 学習率

# 2層ニューラルネットワーク
net = TwoLayerNetwork()

# 学習関数
## Juliaの慣習で、副作用のある（実行ごとに結果が変わる）関数には ! をつける
train!(net::Network) = begin
    # ミニバッチ取得: 対象データ群から batch_size 個のデータを抜き出し
    batch_mask = rand(1:train_size, batch_size)
    batch_x = train_x[batch_mask, :]
    batch_t = train_y[batch_mask, :]
    
    # 勾配の計算
    grad = numeric_gradient(net, batch_x, batch_t)
    
    # パラメータの更新
    net.b[1] -= learning_rate * grad["B1"]
    net.b[2] -= learning_rate * grad["B2"]
    net.w[1] -= learning_rate * grad["W1"]
    net.w[2] -= learning_rate * grad["W2"]
    
    # 学習経過の記録
    loss_value = loss(net, batch_x, batch_t)
    push!(train_loss_list, loss_value)
end

# 学習実行
## iters_num = 1回で 80秒程度かかるため、10,000回実行しようとすると 222時間（≒9日）程度かかる
## (Intel(C) Core i7-7700 3.6 GHz の場合)
@time train!(net)
train_loss_list

 82.919140 seconds (149.05 M allocations: 107.032 GiB, 13.88% gc time)


1-element Array{Any,1}:
 4.916103202232928